In [1]:
import numpy as np
import matplotlib.pyplot as plt
import gymnasium as gym
import gym_quad
import os

from gym_quad.utils.controllers import PI, PID
from mpl_toolkits.mplot3d import Axes3D
from stable_baselines3 import PPO
from utils import *



In [2]:
def run_scen(scenario):
    sim_dfs=[]
    for i in range(2):
        path="./log\Experiment 103\/"+scenario+"/agents/last_model.pkl"
        env = gym.make("PathColav3d-v0", scenario=scenario)
        agent = PPO.load(path)
        sim_df,prog = simulate_environment(env, agent)
        sim_df.to_csv(r'simdata.csv')
        calculate_IAE(sim_df)
        #plot_attitude(sim_df)
        #plot_velocity(sim_df)
        #plot_angular_velocity(sim_df)
        #plot_control_inputs([sim_df])
        #plot_control_errors([sim_df])
        #plot_3d(env, sim_df)
        sim_df['prog']=prog[:-1]
        sim_dfs.append(sim_df)
    return sim_dfs

In [3]:
def stats(sim_dfs,scenario):
    e_means=[]
    e_stds=[]
    h_means=[]
    h_stds=[]
    for t in range(0,10000,1):
        e=[]
        h=[]
        for sim_df in sim_dfs:
            try:
                e.append(sim_df['e'].abs()[t])
            except:
                pass
                #continue#break
            try:
                h.append(sim_df['h'].abs()[t])
            except:
                pass
        if len(e)>0:
            e_means.append(np.mean(e))
            e_stds.append(np.std(e))
            h_means.append(np.mean(h))
            h_stds.append(np.std(h))

        #if t>2000:
        #    print(e)
    e_means=np.array(e_means)
    h_means=np.array(h_means)
    e_stds=np.array(e_stds)
    h_stds=np.array(h_stds)
    t=np.array(range(0,len(e_means),1))/100
    plt.plot(t,e_means)
    plt.fill_between(t,e_means-e_stds,e_means+e_stds,alpha=0.3)
    plt.plot(t,h_means)
    plt.fill_between(t,h_means-h_stds,h_means+h_stds,alpha=0.3)
    plt.grid()
    plt.legend(['e','h'])
    plt.xlabel('Time [s]')
    plt.ylabel('Tracking Error [m]')
    plt.savefig(scenario+'.pdf')

In [4]:
def stats_ape(sim_dfs,scenario):
    err_mean=[]
    err_stds=[]
    for t in range(0,10000,1):
            err=[]
            
            for sim_df in sim_dfs:
                try:
                    err.append(np.sqrt(sim_df['e'][t]**2+sim_df['h'][t]**2))
                except:
                    pass
                
            if len(err)>0:
                err_mean.append(np.mean(err))
                err_stds.append(np.std(err))

    err_mean=np.array(err_mean)
    err_stds=np.array(err_stds)
    t=np.array(range(0,len(err_mean),1))/100
    plt.plot(t,err_mean)
    plt.fill_between(t,err_mean-err_stds,err_mean+err_stds,alpha=0.3)

    plt.grid()
    plt.legend(['Absolute Path Error'])
    plt.xlabel('Time [s]')
    plt.ylabel('Abslute Path Error [m]')
    plt.savefig(scenario+'_ATE.pdf')

In [5]:
# scenarios = ["line","line_new","horizontal_new", "3d_new","intermediate", "proficient", "advanced", "expert"]
# for scenario in scenarios:
#     #sim_dfs=run_scen(scenario)
#     stats_ape(sim_dfs,scenario)

In [6]:
sim_dfs=run_scen('intermediate')

NameNotFound: Environment PathColav3d doesn't exist. 

In [ ]:
def failure_rate(sim_dfs):
    failures=0
    for sim_df in sim_dfs:
        if sim_df['prog'].values[-1]<0.90:
            failures+=1
    return failures/len(sim_dfs)

failure_rate(sim_dfs)

0.5

In [ ]:
# def collision_rate(sim_dfs):
    